This notebook compares different MPCs for the drone with springpendulum example. 

To load cost and timing results from the paper, run the setup and directly load the results (Skip II) and V)

To rerun the experiments on your machine and save your own results file, change the result file names in III) and VI) and run all cells of the notebook.

I) Setup 

In [ ]:
from drone.drone_MPC import DroneMPC, DroneMPCOptions
import numpy as np

# Define an initial state corresponding to hovering.
# For the full model, the state is:
# [y, z, phi, r, theta, y_dot, z_dot, phi_dot, r_dot, theta_dot, w1, w2]
# where:
#   - y = 0.0 (horizontal position), z = 1.0 (altitude)
#   - phi (drone pitch) = 0.0, theta (pendulum angle) = 0.0
#   - r is set to the spring rest length (l0)
#   - All velocities are 0.
#   - Rotor speeds (w1, w2) are chosen such that the total thrust equals the weight.
l0 = 0.3
M = 2.0
m = 0.3
M_total = M + m
c = 1
g = 9.81
# Required total thrust for hovering T = (M_total)*g, and assuming symmetric rotors, each rotor speed is set to T/2 (with c=1)
w_val = (M_total * g) / (2.0*c)
L_rot = 0.2
d = 10.0

# Create an instance of the MPC options (using the updated physical parameters)
opts = DroneMPCOptions(
    M=M,         # drone mass in kg
    m=m,         # load mass in kg (small compared to drone)
    Ixx=0.05,      # moment of inertia in kg·m²
    g=g,        # gravitational acceleration in m/s²
    c=c,         # rotor thrust constant
    L_rot=L_rot,     # half distance between rotors in m
    k=2000.0,       # stiff spring (N/m)
    l0=l0,        # spring rest length in m
    # Constraints and cost weighting matrices remain as defined by default

    N = 300,
    switch_stage = 100,
    step_sizes = [0.01]*300,

    w_min = -2*w_val,
    w_max = 2*w_val,
    w_dot_min = -30,
    w_dot_max = 30,
    phi_min = -np.pi*3/10,
    phi_max = np.pi*3/10,
    F_min = -2*w_val*c,
    F_max = 2*w_val*c,

    integrator_type="IRK",

    create_sim=True # Sim creation code not adapted for parametrized subclass
)

x0 = np.array([
    0.0,    # y position
    0.1,    # z position (hover altitude)
    0.0,    # phi (drone pitch)
    l0,     # r (pendulum length set to rest length)
    0.0,    # theta (pendulum angle)
    0.0,    # y_dot
    0.0,    # z_dot
    0.0,    # phi_dot
    0.0,    # r_dot
    0.0,    # theta_dot
    w_val,  # w1 (rotor speed)
    w_val   # w2 (rotor speed)
])
print("Initial hovering state:", x0)

# Define a reference position far away (for example, [10, 10])
pos_ref = np.array([10.0, 2.0])



In [ ]:
from copy import deepcopy
from drone.utils_drone import simulate_closed_loop_drone
from drone.plotting_utils_drone import plot_drone_mpc_solution
import gc
import shutil

### Setup ###
dt_sim = 0.005 # model with IRK and this stepsize is treated as ground truth here
mpc_opts_sim = deepcopy(opts)
mpc_opts_sim.N = 1
mpc_opts_sim.step_sizes = [dt_sim]*1
mpc_opts_sim.switch_stage = 2
mpc_opts_sim.integrator_type = "IRK"
mpc_sim = DroneMPC(mpc_opts_sim)
sim_solver = mpc_sim.sim_solver_full

del mpc_sim
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

# collect mean costs and solve times
mean_costs = []
mean_solve_times = []
mean_solve_time_per_iter = []

# collect standart deviations
std_dev_costs = []
std_dev_solve_times = []
std_dev_solve_time_per_iter = []

# MPC config shared by all MPCs 
dt_inital_mpc = 0.01
control_step = int(dt_inital_mpc/dt_sim) # determines zero order hold of the MPC
X0 = np.array([0.0, 1.0, 0.0, l0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, w_val, w_val])
duration = 5.0 # was 5.0

# for plotting
save = True
latexify = True

II) Comparison of different predictive Controllers

We treat the model integrated with IRK and a step size of 0.0025 (500 HZ) as ground truth. Details:

0.0) Baseline MPC: Exact model, long horizon and small step sizes, integrated with ERK. We use a levenberg-marquardt regularization of the Hessian of 8e-3 (tuned for appropriate convergence). Levenberg-marquardt can slow down convergence, but helps with problems due to ill-conditioning.

-1.0) Same as 0), but integrated with IRK. Shows IRK not necessary.

1.0) Myopic MPC, Same as 0), but with horizon of 100.

2.0) Exact model case, but with larger step sizes (and applied at a higher frequency). Same time horizon as 0)

3.0) Approximate model only, ignore spring in pendulum model. Despite that, as 0)

4.0) Exact model with exponential increase of step sizes at a sensible rate. Same time lookahead as 0)

5.0) Model switching without step size increase, step sizes and horizon as 0)

6.0) Our approach: Combines model switching with an exponential increase of step size


In [ ]:
### 0) Baseline MPC ###
mpc_opts_baseline = deepcopy(opts)
N = 150
mpc_opts_baseline.N = N
mpc_opts_baseline.step_sizes = [dt_inital_mpc]*N
mpc_opts_baseline.switch_stage = N+1
mpc_opts_baseline.integrator_type = "ERK"
mpc_opts_baseline.levenberg_marquardt = 8e-3
mpc_baseline = DroneMPC(mpc_opts_baseline)

stage_cost_function=mpc_baseline.stage_cost_func_full # save stage costs function

mpc_baseline.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_0, u_traj_0, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_baseline, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_baseline, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_0,
    u_traj=u_traj_0,
    step_pose=10,
    number=0,
    legend=True,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_baseline
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### -1.0) Baseline MPC with IRK ###
mpc_opts_IRK = deepcopy(opts)
N = 150
mpc_opts_IRK.N = N
mpc_opts_IRK.step_sizes = [dt_inital_mpc]*N
mpc_opts_IRK.switch_stage = N+1
mpc_opts_IRK.integrator_type = "IRK"
mpc_opts_IRK.levenberg_marquardt = 8e-3
mpc_IRK = DroneMPC(mpc_opts_IRK)

mpc_IRK.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj, u_traj, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_IRK, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_IRK, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj,
    u_traj=u_traj,
    step_pose=10,
)


del mpc_IRK
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 1) Myopic MPC ###
mpc_opts_1 = deepcopy(opts)
N = 100
mpc_opts_1.N = N
mpc_opts_1.step_sizes = [dt_inital_mpc]*N
mpc_opts_1.switch_stage = N+1
mpc_opts_1.integrator_type = "ERK"
mpc_opts_1.levenberg_marquardt = 8e-3
mpc_1 = DroneMPC(mpc_opts_1)

mpc_1.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_1, u_traj_1, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_1, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_1, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_1,
    u_traj=u_traj_1,
    step_pose=10,
    number=1,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_1
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 2) Larger Step Sizes, Lower Frequency ###
mpc_opts_2 = deepcopy(opts)
N = 75
mpc_opts_2.N = N
mpc_opts_2.step_sizes = [2*dt_inital_mpc]*N
mpc_opts_2.switch_stage = N+1
mpc_opts_2.integrator_type = "ERK"
mpc_opts_2.levenberg_marquardt = 8e-3
mpc_2 = DroneMPC(mpc_opts_2)

mpc_2.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_2, u_traj_2, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_2, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=2*control_step
)

plot_drone_mpc_solution(
    mpc=mpc_2, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_2,
    u_traj=u_traj_2,
    step_pose=10,
    number=2,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_2
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 3)  Approximate model only ###
mpc_opts_3 = deepcopy(opts)
N = 150
mpc_opts_3.N = N
mpc_opts_3.step_sizes = [dt_inital_mpc]*N
mpc_opts_3.switch_stage = 0
mpc_opts_3.integrator_type = "ERK"
mpc_opts_3.levenberg_marquardt = 8e-3
mpc_3 = DroneMPC(mpc_opts_3)

mpc_3.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_3, u_traj_3, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_3, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_3, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_3,
    u_traj=u_traj_3,
    step_pose=10,
    number=3,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_3
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 4) Exponential increase in stepsizes ###
from utils_shared import compute_exponential_step_sizes
mpc_opts_4 = deepcopy(opts)
N = 75
mpc_opts_4.N = N
mpc_opts_4.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=dt_inital_mpc*150,
    N_steps=N,
    plot=False
)
mpc_opts_4.switch_stage = N+1
mpc_opts_4.integrator_type = "ERK"
mpc_opts_4.levenberg_marquardt = 8e-3 
mpc_4 = DroneMPC(mpc_opts_4)

mpc_4.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_4, u_traj_4, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_4, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_4, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_4,
    u_traj=u_traj_4,
    step_pose=10,
    number=4,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_4
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 5) Only model switching ###
mpc_opts_5 = deepcopy(opts)
N = 150
mpc_opts_5.N = N
mpc_opts_5.step_sizes = [dt_inital_mpc]*N
mpc_opts_5.switch_stage = 45
mpc_opts_5.integrator_type = "ERK"
mpc_opts_5.levenberg_marquardt = 8e-3
mpc_5 = DroneMPC(mpc_opts_5)

mpc_5.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_5, u_traj_5, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_5, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_5, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_5,
    u_traj=u_traj_5,
    step_pose=10,
    number=5,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_5
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 6) Ours: Model switching with exponential increase in stepsizes ###
from utils_shared import compute_exponential_step_sizes
mpc_opts_6 = deepcopy(opts)
N = 75
mpc_opts_6.N = N
mpc_opts_6.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=dt_inital_mpc*150,
    N_steps=N,
    plot=True
)
mpc_opts_6.switch_stage = 33
mpc_opts_6.integrator_type = "ERK"
mpc_opts_6.levenberg_marquardt = 8e-3 
mpc_6 = DroneMPC(mpc_opts_6)

mpc_6.set_initial_guess(X0, u_guess=np.zeros(2))

# simulate
x_traj_6, u_traj_6, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
    X0, 
    mpc_6, 
    duration, 
    target_xy=pos_ref,
    stage_cost_function=stage_cost_function,
    sigma_noise=0.0,  
    sim_solver=sim_solver, 
    control_step=control_step
)

plot_drone_mpc_solution(
    mpc=mpc_6, 
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_6,
    u_traj=u_traj_6,
    step_pose=10,
    number=6,
    legend=False,
    save=save,
    latexify=latexify
)

# mean values
mean_costs.append(np.mean(stage_costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(stage_costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_6
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

III) Load or safe results

In [ ]:
# save results or load them

import os
import pickle
from utils_shared import get_dir

data_dir = get_dir("data")
results_file = data_dir / "drone/drone_results.pkl" 

if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        data = pickle.load(f)
    mean_costs = data['mean_costs']
    mean_costs_baseline = mean_costs[0]
    mean_solve_times = data['mean_solve_times']
    mean_solve_time_per_iter = data['mean_solve_time_per_iter']
    std_dev_costs = data['std_dev_costs']
    std_dev_solve_times = data['std_dev_solve_times']
    std_dev_solve_time_per_iter = data['std_dev_solve_time_per_iter']
    x_traj_0 = data['x_traj_0']
    x_traj_1 = data['x_traj_1']
    x_traj_2 = data['x_traj_2']
    x_traj_3 = data['x_traj_3']
    x_traj_4 = data['x_traj_4']
    x_traj_5 = data['x_traj_5']
    x_traj_6 = data['x_traj_6']
else:
    data = {
        'mean_costs': mean_costs,
        'mean_solve_times': mean_solve_times,
        'mean_solve_time_per_iter': mean_solve_time_per_iter,
        'std_dev_costs': std_dev_costs,
        'std_dev_solve_times': std_dev_solve_times,
        'std_dev_solve_time_per_iter': std_dev_solve_time_per_iter,
        'x_traj_0': x_traj_0,
        'x_traj_1': x_traj_1,
        'x_traj_2': x_traj_2,
        'x_traj_3': x_traj_3,
        'x_traj_4': x_traj_4,
        'x_traj_5': x_traj_5,
        'x_traj_6': x_traj_6
    }
    os.makedirs(os.path.dirname(results_file), exist_ok=True)
    with open(results_file, 'wb') as f:
        pickle.dump(data, f)

IV) Plot results 

In [ ]:
# Plotting options for the following plots
latexify=True
save=True

In [ ]:
from drone.plotting_utils_drone import plot_drone_configuration
plot_drone_configuration(
    l0=l0, 
    L=L_rot,
    factor = 1,
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_6,
    step_pose=15,
    number=6,
    legend=False,
    save=save,
    latexify=latexify,
    fontsize=24,
    title="MTS-MPC"
)

In [ ]:
from drone.plotting_utils_drone import plot_drone_configuration
plot_drone_configuration(
    l0=l0, 
    L=L_rot,
    factor = 1,
    reference_xy=pos_ref,
    closed_loop_traj=x_traj_1,
    step_pose=15,
    number=1,
    legend=False,
    save=save,
    latexify=latexify,
    fontsize=24,
    title="Myopic MPC"
)

In [ ]:
from drone.plotting_utils_drone import plot_drone_oscillations
fontsize=40
end = 700
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_0,
    plot_title="Baseline",
    number=0,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
print(dt_sim)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_1,
    plot_title="Myopic MPC",
    number=1,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_2,
    plot_title="Lower frequency",
    number=2,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_3,
    plot_title="Approximate model",
    number=3,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_4,
    plot_title="Increasing step sizes",
    number=4,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_5,
    plot_title="Model switching",
    number=5,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)
plot_drone_oscillations(
    dt_sim=dt_sim,
    closed_loop_traj=x_traj_6,
    plot_title="MTS-MPC (Ours)",
    number=6,                
    save=True,     
    latexify=True,
    fontsize=fontsize,
    end=end      
)

Next, we create bar plots and a pareto frontier of the observed closed-loop costs, solve times and solve times per SQP iteration. 

In [ ]:
print(mean_costs)
print(mean_solve_times)
print(mean_solve_time_per_iter)


print(std_dev_costs)
print(std_dev_solve_times)
print(std_dev_solve_time_per_iter)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import latexify_plot
from plotting_utils_shared import barplot

# Data: mean_costs, mean_solve_times, mean_solve_times_per_iter should be defined already
approach_labels = [
    '0) Baseline',
    '1) Shorter Horizon',
    '2) Larger Step Size',
    '3) Approx. Model',
    '4) Increasing\n    Step Sizes',
    '5) Model Switching',
    '6) Model Switching +\n    Increasing Step Sizes'
]

barplot(
    approach_labels,
    mean_costs,
    mean_solve_times,
    mean_solve_time_per_iter,
    subpath="drone/drone_barplot.pdf",
    latexify=latexify,
    save=save,
    figsize=(8, 10),
    fontsize=12,
)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import pareto_frontier

# distinct symbols per approach
markers = ['D', 's', '^', 'v', 'o', 'X', 'p']

# Marker sizes per approach (indexed same as markers)
marker_sizes = [500, 500, 500, 500, 500, 500, 500]  # customize these

approach_labels = [
    '0) Baseline',
    '1) Shorter Horizon',
    '2) Larger Step Size',
    '3) Approx. Model',
    '4) Increasing\n    Step Sizes',
    '5) Model Switching',
    '6) Model Switching +\n    Increasing Step Sizes'
]
pareto_frontier(
    mean_solve_times,
    mean_costs,
    mean_costs_baseline,
    approach_labels,
    markers,
    marker_sizes,
    subpath="drone/pareto_front_drone.pdf",
    latexify=latexify,
    save=save,
    figsize=(7.5, 6),
    fontsize=25,
    x_lim=None,
    x_nonlog=False,
    y_nonlog=True,
    legend=False
)


V) Sweep over switching stage

In [ ]:
from utils_shared import compute_exponential_step_sizes
from copy import deepcopy

### sweep
mpc_opts_sweep = deepcopy(opts)
N = 75
mpc_opts_sweep.N = N
mpc_opts_sweep.step_sizes = compute_exponential_step_sizes(
    dt_initial=dt_inital_mpc,
    T_total=dt_inital_mpc*150,
    N_steps=N,
    plot=True
)
mpc_opts_sweep.integrator_type = "ERK"
mpc_opts_sweep.levenberg_marquardt = 8e-3 

switching_indices = list(range (1, N))
mean_costs_sweep = []
for switching_index in switching_indices:
    mpc_opts_sweep.switch_stage = switching_index
    mpc_sweep = DroneMPC(mpc_opts_sweep)

    mpc_sweep.set_initial_guess(X0, u_guess=np.zeros(2))

    # simulate
    _, _, stage_costs, solve_times, SQP_iters = simulate_closed_loop_drone(
        X0, 
        mpc_sweep, 
        duration, 
        target_xy=pos_ref,
        stage_cost_function=stage_cost_function,
        sigma_noise=0.0,  
        sim_solver=sim_solver, 
        control_step=control_step
    )

    del mpc_sweep
    gc.collect()
    shutil.rmtree('c_generated_code', ignore_errors=True)

    # mean values
    mean_costs_sweep.append(np.mean(stage_costs))



VI) Safe or load sweep results and plot them

In [ ]:
from utils_shared import get_dir
import pickle
import os
data_dir = get_dir("data")
results_file_sweep = data_dir / "drone/drone_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep = data['mean_costs_sweep']
    switching_indices = data['switching_indices']
    mean_costs_baseline = data['mean_costs_baseline']
else:
    data = {
        'mean_costs_sweep': mean_costs_sweep,
        'switching_indices': switching_indices,
        'mean_costs_baseline': mean_costs_baseline
    }
    os.makedirs(os.path.dirname(results_file_sweep), exist_ok=True)
    with open(results_file_sweep, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(8,5))
plt.plot(switching_indices[1:], 100*np.array(mean_costs_sweep[1:]-mean_costs_baseline)/mean_costs_baseline, marker='o', linestyle='-', linewidth=2)

plt.xlabel("Switching Index", fontsize=12)
plt.ylabel("Mean Closed Loop Cost Increase in %", fontsize=12)
plt.title("Mean Closed Loop Costs vs. Switching Index", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.yscale('log')
plt.show()